In [1]:
# load in libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
#TODO: apply labels to the processed EEG data using the annotations from before


Load in data

In [2]:
data_path = '../processed_EEG_data\clean_2025-02-23_14-24-48_1.csv'

data = pd.read_csv(data_path)

Model
